THE CODE BELOW INTEGRATES THE HW4 STUFF AND ADDS THE WORK FROM ABOVE...

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
path = 'inputs/'

In [13]:
def insert_holidays(testy):
    data = testy.copy()
    data['nat_holiday'] = 0
    data['reg_holiday'] = 0
    data['loc_holiday'] = 0

    stores = stores_new.copy()
    holidays = pd.read_csv(path + 'holidays_events.csv',
                        usecols=['date', 'type', 'locale', 'locale_name', 'transferred'],
                        dtype={'type': 'category', 'locale': 'category', 'locale_name': 'category', 'transferred': 'bool'},
                        parse_dates=['date'], infer_datetime_format=True
                        )
    holidays = holidays.reset_index()
    stores = stores.reset_index()

    for index, row in holidays.iterrows():
        store_nbr_list = []
        if row['locale'] == 'Local':
            for sindex, srow in stores.iterrows():
                if srow['city'] == row['locale_name']:
                    data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'loc_holiday'] = 1
        if row['locale'] == 'Regional':
            for sindex, srow in stores.iterrows():
                if srow['state'] == row['locale_name']:
                    data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'reg_holiday'] = 1
        if row['locale'] == 'National':
            data.loc[data['date'] == row['date'], 'nat_holiday'] = 1

    return data

In [14]:
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True)

########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(7).mean()

# Create continguous moving average of oil prices
calendar = pd.DataFrame()
calendar['date'] = pd.date_range('2013-01-01', '2017-08-31')
# calendar.head()
########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# as calendar.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
cally = pd.merge(calendar, data_oil, how='outer', on='date' )
calendar = cally.copy()
# calendar = calendar.merge(data_oil, on='date')

# calendar.head()

data_oil['ma_oil'].fillna(method='ffill', inplace=True)
calendar['ma_oil'].fillna(method='ffill', inplace=True)
calendar['wd'] = [i.dayofweek < 5 for i in calendar['date']]

Import Stores Data

In [15]:
stores_new = pd.read_csv(path + 'stores.csv',
                         usecols=['store_nbr', 'city', 'state', 'type', 'cluster'],
                         dtype={'store_nbr': 'category', 'city': 'category', 'state': 'category', 'type': 'category', 'cluster': 'category'})

Import Transactions Data

In [16]:
transactions = pd.read_csv(path + 'transactions.csv',
                          usecols=['store_nbr', 'date', 'transactions'],
                          dtype={'store_nbr': 'category', 'transactions': 'float32'},
                          parse_dates=['date'], infer_datetime_format=True)
# transactions.date = transactions.date.dt.to_period('D')

In [17]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32', 'onpromotion': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

Merge Training Data

In [20]:
train_trans = pd.merge(transactions, df_train, how='outer', on =['store_nbr', 'date'])
train_trans_stores = train_trans.merge(stores_new, on='store_nbr')
train_trans_stores_oil = train_trans_stores.merge(calendar, on='date')
data_train = insert_holidays(train_trans_stores_oil)
data_train.transactions.fillna(0, inplace=True)
data_train['ma_oil'].fillna(method='ffill', inplace=True)
data_train['date'] = pd.to_datetime(data_train['date'])
data = data_train.copy()
data['date'] = pd.to_datetime(data['date'])

In [21]:
pd.to_datetime('2017-04-01') in data.date.unique()


True

Load Test Data

In [22]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date', 'onpromotion'],
                      dtype={'store_nbr': 'category', 'family': 'category', 'onpromotion': 'float32'},
                      parse_dates=['date'], infer_datetime_format=True)
test_stores = df_test.merge(stores_new, on='store_nbr')
test_stores_cal = test_stores.merge(calendar, on='date')
data_test = insert_holidays(test_stores_cal)

BELOW IS THE FIRST ATTEMPT TO USE LightGBM

In [23]:
# set the range of data used in training
sdate = '2016-04-01'
edate = '2017-08-15'

data_cut_dates = data.loc[(data['date'] >= sdate) & (data['date'] <= edate)]
data_cut_dates

,date,store_nbr,transactions,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,ma_oil,wd,nat_holiday,reg_holiday,loc_holiday
2102760,2016-04-01,25,1237.0,AUTOMOTIVE,1.0,0.0,Salinas,Santa Elena,D,1,35.36,39.407143,True,0,0,0
2102761,2016-04-01,25,1237.0,BABY CARE,0.0,0.0,Salinas,Santa Elena,D,1,35.36,39.407143,True,0,0,0
2102762,2016-04-01,25,1237.0,BEAUTY,5.0,0.0,Salinas,Santa Elena,D,1,35.36,39.407143,True,0,0,0
2102763,2016-04-01,25,1237.0,BEVERAGES,4318.0,12.0,Salinas,Santa Elena,D,1,35.36,39.407143,True,0,0,0
2102764,2016-04-01,25,1237.0,BOOKS,0.0,0.0,Salinas,Santa Elena,D,1,35.36,39.407143,True,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2016-10-26,52,0.0,POULTRY,0.0,0.0,Manta,Manabi,A,11,48.75,50.170000,True,0,0,0
3000884,2016-10-26,52,0.0,PREPARED FOODS,0.0,0.0,Manta,Manabi,A,11,48.75,50.170000,True,0,0,0
3000885,2016-10-26,52,0.0,PRODUCE,0.0,0.0,Manta,Manabi,A,11,48.75,50.170000,True,0,0,0
3000886,2016-10-26,52,0.0,SCHOOL AND OFFICE SUPPLIES,0.0,0.0,Manta,Manabi,A,11,48.75,50.170000,True,0,0,0


In [24]:
# Train data
X = np.zeros((data_cut_dates.family.nunique(), data_cut_dates.date.nunique(), 4))
print(f'x shape: {X.shape}')

# Truth data
y_truth = np.zeros(X.shape[:2])
print(f'y shape: {y_truth.shape}')

family_groups = data_cut_dates.groupby('family')
for i, (family, family_group) in enumerate(family_groups):
    for j, (date, group) in enumerate(family_group.groupby('date')):
        if i == 1 and j == 1:
            first = False
        X[i][j][0] = group.onpromotion.sum()
        X[i][j][1] = group.ma_oil.mean()
        X[i][j][2] = group.wd.any()
        X[i][j][3] = min(group.nat_holiday.sum(), 1)
        y_truth[i][j] = group.sales.sum()

x shape: (33, 501, 4)
y shape: (33, 501)


LightGBM -- first pass

In [ ]:
# from lightgbm import LGBMRegressor
#
# #Can add weights/grouping to this as well if we want
# def RMSLE(y_pred, y):
#     is_higher_better = False
#     labels = y.get_label()
#     loss = np.sum((np.log(1 + y_pred) - np.log(1 + labels))**2)
#    # for i in range(len(y_pred)):
#   #      loss += (np.log(1 + y_pred[i]) - np.log(1 + labels[i]))**2
#     loss = loss / len(y_pred)
#     return "RMSLE", loss, is_higher_better
# y_pred = np.zeros(X.shape[:2])
# models = []
#
# for i, (x, y) in enumerate(zip(X, y_truth)):
#     model = LGBMRegressor(n_estimators=100)
#     models.append(model)
#     model.fit(x, y, eval_metric=RMSLE)
#     # y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)
#     y_pred[i] = model.predict(x)
#     y_pred[i][np.where(y_pred[i] < 0)] = 0

In [ ]:
# loss = np.sum( ( np.log(y_pred + 1) + np.log(y_truth + 1) )**2, axis=1 ) / y_truth.shape[1]

In [ ]:
# loss

In [ ]:
# X_test = np.zeros((data_cut_dates.family.nunique(), 16, 4))
#
# family_groups = data_test.groupby('family')
# for i, (family, family_group) in enumerate(family_groups):
#     for j, (date, group) in enumerate(family_group.groupby('date')):
#         X_test[i][j][0] = group.onpromotion.sum()
#         X_test[i][j][1] = group.ma_oil.mean()
#         X_test[i][j][2] = group.wd.any()
#         X_test[i][j][3] = min(group.nat_holiday.sum(), 1)

In [ ]:
# test_pred = []
# for i, model in enumerate(models):
#     test_pred.append(model.predict(X_test[i]))

In [ ]:
# submission_array = np.zeros((16, data_cut_dates.store_nbr.nunique(), data_cut_dates.family.nunique()))
#
# for i in range(16):
#     for j in range(data_cut_dates.store_nbr.nunique()):
#         for k in range(data_cut_dates.family.nunique()):
#             submission_array[i][j][k] = test_pred[k][i]


In [ ]:
# submission_array.flatten().shape

In [ ]:
# # Create submission
# df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
# df_sub.sales = submission_array.flatten()
# df_sub.to_csv('submission.csv', index=True)

In [25]:
data_cut_dates.family.nunique()

33

In [26]:
data = data.sort_values(['date', 'store_nbr', 'family'])

In [27]:
data

,date,store_nbr,transactions,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,ma_oil,wd,nat_holiday,reg_holiday,loc_holiday
33,2013-01-01,1,0.0,AUTOMOTIVE,0.000000,0.0,Quito,Pichincha,D,13,NaN,NaN,True,1,0,0
34,2013-01-01,1,0.0,BABY CARE,0.000000,0.0,Quito,Pichincha,D,13,NaN,NaN,True,1,0,0
35,2013-01-01,1,0.0,BEAUTY,0.000000,0.0,Quito,Pichincha,D,13,NaN,NaN,True,1,0,0
36,2013-01-01,1,0.0,BEVERAGES,0.000000,0.0,Quito,Pichincha,D,13,NaN,NaN,True,1,0,0
37,2013-01-01,1,0.0,BOOKS,0.000000,0.0,Quito,Pichincha,D,13,NaN,NaN,True,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876473,2017-08-15,9,2155.0,POULTRY,438.132996,0.0,Quito,Pichincha,B,6,47.57,48.648571,True,0,0,0
2876474,2017-08-15,9,2155.0,PREPARED FOODS,154.552994,1.0,Quito,Pichincha,B,6,47.57,48.648571,True,0,0,0
2876475,2017-08-15,9,2155.0,PRODUCE,2419.729004,148.0,Quito,Pichincha,B,6,47.57,48.648571,True,0,0,0
2876476,2017-08-15,9,2155.0,SCHOOL AND OFFICE SUPPLIES,121.000000,8.0,Quito,Pichincha,B,6,47.57,48.648571,True,0,0,0


In [28]:
drop_data = data_cut_dates[['onpromotion', 'ma_oil', 'wd', 'nat_holiday', 'family', 'sales', 'store_nbr']]

In [29]:
drop_data

,onpromotion,ma_oil,wd,nat_holiday,family,sales,store_nbr
2102760,0.0,39.407143,True,0,AUTOMOTIVE,1.0,25
2102761,0.0,39.407143,True,0,BABY CARE,0.0,25
2102762,0.0,39.407143,True,0,BEAUTY,5.0,25
2102763,12.0,39.407143,True,0,BEVERAGES,4318.0,25
2102764,0.0,39.407143,True,0,BOOKS,0.0,25
...,...,...,...,...,...,...,...
3000883,0.0,50.170000,True,0,POULTRY,0.0,52
3000884,0.0,50.170000,True,0,PREPARED FOODS,0.0,52
3000885,0.0,50.170000,True,0,PRODUCE,0.0,52
3000886,0.0,50.170000,True,0,SCHOOL AND OFFICE SUPPLIES,0.0,52


In [30]:
store_array = []
for i in drop_data['store_nbr'].unique():
    store_array.append(drop_data.loc[drop_data['store_nbr'] == i])

In [31]:
store_array[0]

,onpromotion,ma_oil,wd,nat_holiday,family,sales,store_nbr
2102760,0.0,39.407143,True,0,AUTOMOTIVE,1.0,25
2102761,0.0,39.407143,True,0,BABY CARE,0.0,25
2102762,0.0,39.407143,True,0,BEAUTY,5.0,25
2102763,12.0,39.407143,True,0,BEVERAGES,4318.0,25
2102764,0.0,39.407143,True,0,BOOKS,0.0,25
...,...,...,...,...,...,...,...
2999134,0.0,50.170000,True,0,POULTRY,0.0,25
2999135,0.0,50.170000,True,0,PREPARED FOODS,0.0,25
2999136,0.0,50.170000,True,0,PRODUCE,0.0,25
2999137,0.0,50.170000,True,0,SCHOOL AND OFFICE SUPPLIES,0.0,25


In [32]:
fam_array = []
for store_df in store_array:
    for i in store_df['family'].unique():
        fam_array.append(store_df.loc[store_df['family'] == i])

In [33]:
fam_array[33]

,onpromotion,ma_oil,wd,nat_holiday,family,sales,store_nbr
2102793,0.0,39.407143,True,0,AUTOMOTIVE,8.0,1
2104575,0.0,39.407143,False,0,AUTOMOTIVE,6.0,1
2106357,0.0,39.407143,False,0,AUTOMOTIVE,1.0,1
2108139,0.0,39.407143,True,0,AUTOMOTIVE,5.0,1
2109921,0.0,36.132857,True,0,AUTOMOTIVE,4.0,1
...,...,...,...,...,...,...,...
2992011,0.0,50.514286,False,0,AUTOMOTIVE,6.0,1
2993793,0.0,50.514286,False,0,AUTOMOTIVE,0.0,1
2995575,0.0,50.472857,True,0,AUTOMOTIVE,3.0,1
2997357,0.0,50.344286,True,0,AUTOMOTIVE,8.0,1


In [34]:
from sklearn.model_selection import train_test_split
X_trains = []
X_valids = []
y_trains = []
y_valids = []
for df in fam_array:
    X_train, X_valid, y_train, y_valid = train_test_split(df, df.loc[:, 'sales'].squeeze(), test_size=16, shuffle=False)
    X_trains.append(X_train)
    X_valids.append(X_valid)
    y_trains.append(y_train)
    y_valids.append(y_valid)

In [47]:
X_trains[1].family.unique()[0]

'BABY CARE'

In [36]:
X_valids[0]

,onpromotion,ma_oil,wd,nat_holiday,family,sales,store_nbr
2972376,0.0,49.700000,True,0,AUTOMOTIVE,0.0,25
2974158,0.0,49.891429,True,0,AUTOMOTIVE,0.0,25
2975940,0.0,50.148571,True,0,AUTOMOTIVE,0.0,25
2977722,0.0,50.234286,True,0,AUTOMOTIVE,0.0,25
2979504,0.0,50.234286,False,0,AUTOMOTIVE,0.0,25
2981286,0.0,50.234286,False,0,AUTOMOTIVE,0.0,25
2983068,0.0,50.167143,True,0,AUTOMOTIVE,0.0,25
2984850,0.0,50.244286,True,0,AUTOMOTIVE,0.0,25
2986632,0.0,50.505714,True,0,AUTOMOTIVE,0.0,25
2988414,0.0,50.447143,True,0,AUTOMOTIVE,0.0,25


In [37]:
y_trains

[2102760    1.0
 2104542    4.0
 2106324    5.0
 2108106    3.0
 2109888    1.0
           ... 
 2963466    0.0
 2965248    0.0
 2967030    0.0
 2968812    0.0
 2970594    0.0
 Name: sales, Length: 485, dtype: float32,
 2102761    0.0
 2104543    0.0
 2106325    0.0
 2108107    0.0
 2109889    0.0
           ... 
 2963467    0.0
 2965249    0.0
 2967031    0.0
 2968813    0.0
 2970595    0.0
 Name: sales, Length: 485, dtype: float32,
 2102762    5.0
 2104544    4.0
 2106326    4.0
 2108108    2.0
 2109890    3.0
           ... 
 2963468    0.0
 2965250    0.0
 2967032    0.0
 2968814    0.0
 2970596    0.0
 Name: sales, Length: 485, dtype: float32,
 2102763    4318.0
 2104545    3752.0
 2106327    2225.0
 2108109    2087.0
 2109891    1547.0
             ...  
 2963469       0.0
 2965251       0.0
 2967033       0.0
 2968815       0.0
 2970597       0.0
 Name: sales, Length: 485, dtype: float32,
 2102764    0.0
 2104546    0.0
 2106328    0.0
 2108110    0.0
 2109892    0.0
           

In [76]:
count = 0
names = []


for i, (x, y, x_valid, y_valid) in enumerate(zip(X_trains, y_trains, X_valids, y_valids)):
    save_to = 'store_{}_{}.dat'.format(x.store_nbr.unique()[0],x.family.unique()[0] )
    save_to = save_to.replace(" ", "_")
    save_to = save_to.replace("/", "-")
    # print(save_to)
    names.append(save_to)
    count += 1

names
# len(names)
# print(count)

['store_25_AUTOMOTIVE.dat',
 'store_25_BABY_CARE.dat',
 'store_25_BEAUTY.dat',
 'store_25_BEVERAGES.dat',
 'store_25_BOOKS.dat',
 'store_25_BREAD-BAKERY.dat',
 'store_25_CELEBRATION.dat',
 'store_25_CLEANING.dat',
 'store_25_DAIRY.dat',
 'store_25_DELI.dat',
 'store_25_EGGS.dat',
 'store_25_FROZEN_FOODS.dat',
 'store_25_GROCERY_I.dat',
 'store_25_GROCERY_II.dat',
 'store_25_HARDWARE.dat',
 'store_25_HOME_AND_KITCHEN_I.dat',
 'store_25_HOME_AND_KITCHEN_II.dat',
 'store_25_HOME_APPLIANCES.dat',
 'store_25_HOME_CARE.dat',
 'store_25_LADIESWEAR.dat',
 'store_25_LAWN_AND_GARDEN.dat',
 'store_25_LINGERIE.dat',
 'store_25_LIQUOR,WINE,BEER.dat',
 'store_25_MAGAZINES.dat',
 'store_25_MEATS.dat',
 'store_25_PERSONAL_CARE.dat',
 'store_25_PET_SUPPLIES.dat',
 'store_25_PLAYERS_AND_ELECTRONICS.dat',
 'store_25_POULTRY.dat',
 'store_25_PREPARED_FOODS.dat',
 'store_25_PRODUCE.dat',
 'store_25_SCHOOL_AND_OFFICE_SUPPLIES.dat',
 'store_25_SEAFOOD.dat',
 'store_1_AUTOMOTIVE.dat',
 'store_1_BABY_CARE.dat'

In [65]:
def rmsle_lgbm(y_pred, data):

    y_true = np.array(data.get_label())
    score = np.sqrt(np.mean(np.power(np.log1p(y_true) - np.log1p(y_pred), 2)))

    return 'rmsle', score, False

In [78]:
from tqdm.notebook import tqdm
import pickle
import lightgbm as lgb
import warnings

warnings.filterwarnings('ignore')
models = []
y_pred = np.zeros((len(X_trains), X_trains[0].shape[0]))
for i, (x, y, x_valid, y_valid) in tqdm(enumerate(zip(X_trains, y_trains, X_valids, y_valids))):
    lgb_train = lgb.Dataset(x, y)
    lgb_eval = lgb.Dataset(x_valid, y_valid)
    params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'metric': 'custom',
    'num_leaves': 10,
    'learning_rate': 0.05,
    'verbose': -1
    }
    model = lgb.train(params,
                      train_set=lgb_train,
                      valid_sets=lgb_eval,
                      feval=rmsle_lgbm,
                      callbacks=[lgb.early_stopping(stopping_rounds=30)],
                      num_boost_round=100,
                      verbose_eval=False)
    models.append(model)
    save_to = 'store_{}_{}.dat'.format(x.store_nbr.unique()[0],x.family.unique()[0] )
    save_to = save_to.replace(" ", "_")
    save_to = save_to.replace("/", "-")
    save_to = "models/" + save_to
    pickle.dump(model, open(save_to, 'wb'))
    model.fit(x, y, eval_metric=rmsle_lgbm)
    y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)
    y_pred[i] = model.predict(x)
    y_pred[i][np.where(y_pred[i] < 0)] = 0

0it [00:00, ?it/s]

Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmsle: 0.0185424


C:\Users\jarret\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\jarret\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\jarret\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


AttributeError: 'Booster' object has no attribute 'fit'